In [1]:
import sys,os,os.path

sys.path.append("../../")   # cite IC from parent directory
                            # NOTE if you can't import IC stuff, its because of the
                            # above line
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

import matplotlib.pyplot as plt
import pandas as pd
import numpy  as np
import tables as tb
import IC.invisible_cities.io.dst_io                           as     dstio
import IC.invisible_cities.io.mcinfo_io as mcio
from    IC.invisible_cities.core.core_functions   import shift_to_bin_centers
import iminuit,probfit

import scipy.special as special
from scipy.stats import skewnorm
from scipy.optimize import curve_fit

from scipy.integrate import quad

from Tl_studies.functions import isaura_analyse as isa

### This notepad only exists because I forgot to collect efficiency errors earlier

In [6]:
print("Efficiency error port 1a:")

er_1a = isa.ratio_error(1.22, 1218, 100009)
print(er_1a)
er_1b = isa.ratio_error(1.29, 1234, 95548)
print(er_1b)
er_2a = isa.ratio_error(0.97, 688, 71041)
print(er_2a)
er_2b = isa.ratio_error(0.90, 858, 95437)
print(er_2b)

Efficiency error port 1a:
0.035169389312764573
0.03695886505454724
0.03715954350858617
0.030863298890162583


In [12]:
# lets speed this up a little
port1a_events = [54864, 39361, 19622, 11009, 1218]
cut_type = ['Satellite Cut', 'Fiducial', 'One track', 'overlap', 'ROI']

def efficiency_errors(events, tot_events, cut_type):

    for i in range(len(events)):
        if (i==0):
            print(cut_type[i])
            # calculate efficiency first
            effic = (events[i]/tot_events)*100
            
            error = isa.ratio_error(effic, events[i], tot_events)
            print(effic, error)
        else:
            print(cut_type[i])

            # calculate efficiency (relative)
            effic = (events[i]/events[i-1])*100

            error = isa.ratio_error(effic, events[i], events[i-1])
            print(effic, error)


In [13]:
efficiency_errors(port1a_events, 100009, cut_type)

Satellite Cut
54.859062684358406 0.2914559745103282
Fiducial
71.74285505978419 0.473898062551036
One track
49.851375727242704 0.4356481552531854
overlap
56.10539190704311 0.6680971458838539
ROI
11.063675174856936 0.33408870467384494


In [15]:
port1b_events = [57212, 40209, 20933, 12043, 1234]

efficiency_errors(port1b_events, 95548, cut_type)

Satellite Cut
59.87775777619626 0.3165308095598928
Fiducial
70.280710340488 0.4573590033202972
One track
52.060483971250214 0.4437114630364524
overlap
57.53117087851718 0.6579897377136344
ROI
10.24661629162169 0.30627075416357386


In [17]:
port2a_events = [32727, 23871, 11998, 6734, 688]

efficiency_errors(port2a_events, 71041, cut_type)

Satellite Cut
46.067763685758926 0.3077664125400224
Fiducial
72.93977449812081 0.6208355167754745
One track
50.261823970508146 0.5624813718685994
overlap
56.12602100350058 0.8546048323091722
ROI
10.216810216810217 0.4089262840351249


In [18]:
port2b_events = [45891, 33096, 16083, 8824, 858]

efficiency_errors(port2b_events, 95437, cut_type)

Satellite Cut
48.085124218070554 0.27315093948517566
Fiducial
72.11871608812186 0.5200849439199199
One track
48.59499637418419 0.4671002674360337
overlap
54.86538581110489 0.7268467739142851
ROI
9.723481414324569 0.3477186092455814


In [25]:
# doing the full error system here instead
def binom_efficiency_errors(events, tot_events, cut_type):
    for i in range(len(events)):
        if (i==0):
            print(cut_type[i])
            epsilon = events[i] / tot_events
            # number of events within standard deviation of the number of passing events
            error_efficiency = np.sqrt((epsilon*(1-epsilon))*tot_events)

            # getting error on efficiency from this:
            error = 100*(error_efficiency/tot_events)
            print(epsilon * 100, error)
        else:
            print(cut_type[i])
            # relative
            epsilon = events[i] / events[i-1]
            # number of events within standard deviation of the number of passing events
            error_efficiency = np.sqrt((epsilon*(1-epsilon))*events[i-1])

            # getting error on efficiency from this:
            error = 100*(error_efficiency/events[i-1])
            print(epsilon * 100, error)

In [26]:
binom_efficiency_errors(port1a_events, 100009, cut_type)

Satellite Cut
54.859062684358406 0.15735840158156605
Fiducial
71.74285505978419 0.19222493982490932
One track
49.851375727242704 0.25202000968779276
overlap
56.10539190704311 0.3542715295759704
ROI
11.063675174856936 0.29896156745119534


In [28]:
binom_efficiency_errors(port1b_events, 95548, cut_type)

Satellite Cut
59.87775777619626 0.15856763974972476
Fiducial
70.280710340488 0.1910705345493964
One track
52.060483971250214 0.2491376060799884
overlap
57.53117087851718 0.34164180954707696
ROI
10.24661629162169 0.27634289865371964


In [29]:
binom_efficiency_errors(port2a_events, 71041, cut_type)

Satellite Cut
46.067763685758926 0.18701147108997798
Fiducial
72.93977449812081 0.24558102948729998
One track
50.261823970508146 0.3236150744988559
overlap
56.12602100350058 0.4530344309078363
ROI
10.216810216810217 0.3690783942525689


In [30]:
binom_efficiency_errors(port2b_events, 95437, cut_type)

Satellite Cut
48.085124218070554 0.16173085824514208
Fiducial
72.11871608812186 0.20932283928738124
One track
48.59499637418419 0.2747329314291579
overlap
54.86538581110489 0.39239237089392043
ROI
9.723481414324569 0.31540280097135814
